In [8]:
class Node:
    def __init__(self, node_type, value=None, left=None, right=None):
        self.type = node_type  # 'operator' or 'operand'
        self.value = value      # Only for operand nodes (e.g., 'age > 30')
        self.left = left        # Left child
        self.right = right      # Right child

    def __repr__(self):
        return f"Node(type={self.type}, value={self.value}, left={self.left}, right={self.right})"


In [9]:
import re

# Helper function to parse condition into operand nodes
def parse_condition(condition):
    match = re.match(r"(\w+)\s*(>|>=|<|<=|==|!=)\s*(\d+|'[\w\s]+')", condition)
    if match:
        field, operator, value = match.groups()
        value = value.strip("'")  # Remove quotes from string values
        return Node("operand", value={"field": field, "operator": operator, "value": value})
    raise ValueError(f"Invalid condition: {condition}")

# Convert rule string into AST (create_rule)
def create_rule(rule_string):
    rule_string = rule_string.replace("AND", "&&").replace("OR", "||")  # Use Python-friendly operators
    tokens = re.split(r'(\(|\)|&&|\|\|)', rule_string)
    tokens = [token.strip() for token in tokens if token.strip()]

    def parse_expression(tokens):
        if not tokens:
            return None

        stack = []
        while tokens:
            token = tokens.pop(0)
            if token == '(':
                stack.append(parse_expression(tokens))
            elif token == ')':
                break
            elif token in ['&&', '||']:
                right = stack.pop()
                left = stack.pop()
                operator_node = Node("operator", token, left, right)
                stack.append(operator_node)
            else:
                stack.append(parse_condition(token))
        return stack[0] if stack else None

    return parse_expression(tokens)


In [10]:
# Combine multiple ASTs into one with efficiency in mind
def combine_rules(rules):
    combined_root = None
    for rule_string in rules:
        rule_ast = create_rule(rule_string)
        if combined_root is None:
            combined_root = rule_ast
        else:
            # Combine the new rule with the previous ones using AND (&&)
            combined_root = Node("operator", "&&", combined_root, rule_ast)
    return combined_root


In [11]:
def evaluate_operand(node, data):
    field = node.value["field"]
    operator = node.value["operator"]
    value = node.value["value"]
    
    if isinstance(data[field], str):
        value = str(value)
    else:
        value = float(value)
    
    if operator == '>':
        return data[field] > value
    elif operator == '>=':
        return data[field] >= value
    elif operator == '<':
        return data[field] < value
    elif operator == '<=':
        return data[field] <= value
    elif operator == '==':
        return data[field] == value
    elif operator == '!=':
        return data[field] != value
    else:
        raise ValueError(f"Invalid operator {operator}")

def evaluate_rule(ast, data):
    if ast.type == "operand":
        return evaluate_operand(ast, data)
    elif ast.type == "operator":
        if ast.value == '&&':
            return evaluate_rule(ast.left, data) and evaluate_rule(ast.right, data)
        elif ast.value == '||':
            return evaluate_rule(ast.left, data) or evaluate_rule(ast.right, data)
    return False


In [12]:
# Validate input attributes against a defined schema
VALID_ATTRIBUTES = {"age", "department", "salary", "experience"}

def validate_data(data):
    for key in data:
        if key not in VALID_ATTRIBUTES:
            raise ValueError(f"Invalid attribute: {key}")
    return True

def validate_rule_string(rule_string):
    try:
        create_rule(rule_string)  # Try creating the AST
    except ValueError as ve:
        raise ValueError(f"Invalid rule string: {rule_string}. Error: {ve}")


In [13]:
def modify_rule(node, field, new_value, operator=None):
    if node.type == "operand" and node.value["field"] == field:
        node.value["value"] = new_value
        if operator:
            node.value["operator"] = operator
    if node.left:
        modify_rule(node.left, field, new_value, operator)
    if node.right:
        modify_rule(node.right, field, new_value, operator)

# Example: modify salary comparison in a rule to '> 60000'
# modify_rule(ast, 'salary', 60000, '>')


In [15]:
# Test Case 1: Create and verify AST
rule1 = "((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing')) AND (salary > 50000 OR experience > 5)"
ast1 = create_rule(rule1)
print(ast1)

# Test Case 2: Combine rules
rule2 = "((age > 30 AND department = 'Marketing')) AND (salary > 20000 OR experience > 5)"
combined_ast = combine_rules([rule1, rule2])  # Ensure you implement this function
print(combined_ast)

# Test Case 3: Evaluate rule
data = {"age": 35, "department": "Sales", "salary": 60000, "experience": 3}
print(evaluate_rule(ast1, data))  # Expected: True

# Test Case 4: Modify rule
modify_rule(ast1, 'salary', 70000, '>')  # Ensure you implement this function
print(evaluate_rule(ast1, data))  # Expected: False after modification

# Test Case 5: Error handling
try:
    invalid_rule = "age > 30 AND"
    validate_rule_string(invalid_rule)  # Ensure you implement this function
except ValueError as e:
    print(e)


IndexError: pop from empty list